In [2]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import sys
import numpy as np
import pandas as pd
import regex as re
import requests
import lxml
from lxml.html.soupparser import fromstring
import prettify
import numbers
import htmltext

In [16]:
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}
with requests.Session() as s:
    #city = 'dallas'
    
    base_url = 'https://www.zillow.com/homes/for_sale/'
    search_url = '?searchQueryState=%7B"pagination"%3A%7B%7D%2C"mapBounds"%3A%7B"west"%3A-97.32331467871094%2C"east"%3A-96.2439079404297%2C"south"%3A32.782893111556746%2C"north"%3A33.457573709558645%7D%2C"customRegionId"%3A"ed70e61b24X1-CRlk9c6vnwe2ke_xlf96"%2C"isMapVisible"%3Afalse%2C"filterState"%3A%7B"ah"%3A%7B"value"%3Atrue%7D%2C"sort"%3A%7B"value"%3A"globalrelevanceex"%7D%2C"price"%3A%7B"max"%3A500000%7D%2C"mp"%3A%7B"max"%3A1666%7D%7D%2C"isListVisible"%3Atrue%7D'
    url = base_url + search_url
    r = s.get(url, headers=req_headers)
    soup = BeautifulSoup(r.content, 'html.parser')

    
    import math
    t = str(soup.title)
    home_count = int(re.split(r'Homes For Sale - ', t)[1].split(' ')[0])
    pages = math.ceil(home_count/40)    

    for i in range(2,pages):
        print(i)
   

2
3
4
5
6
7


In [ ]:
 
    url2 = base_url+'2_p/' + search_url
    url3 = base_url+'3_p/' + search_url
    url4 = base_url+'4_p/' + search_url
    url5 = base_url+'5_p/' + search_url
    url6 = base_url+'6_p/' + search_url
    url7 = base_url+'7_p/' + search_url
    url8 = base_url+'8_p/' + search_url
    url9 = base_url+'9_p/' + search_url
    url10 = base_url+'10_p/' + search_url



    r = s.get(url, headers=req_headers)
    r2 = s.get(url2, headers=req_headers)
    r3 = s.get(url3, headers=req_headers)
    r4 = s.get(url4, headers=req_headers)
    r5 = s.get(url5, headers=req_headers)
    r6 = s.get(url6, headers=req_headers)
    r7 = s.get(url7, headers=req_headers)
    r8 = s.get(url8, headers=req_headers)
    r9 = s.get(url9, headers=req_headers)
    r10 = s.get(url10, headers=req_headers)
    
    url_links = [url, url2, url3, url4, url5, url6, url7, url8, url9, url10]
    
    #print(url_links)

In [17]:
def web_scrape(soup):
    #all for loops are pulling the specified variable using beautiful soup and inserting into said variable
    df2 = pd.DataFrame()
    address = soup.find_all (class_= 'list-card-addr')
    for i in range(len(address)):
        address[i] = str(address[i]).split('<address class="list-card-addr">')[1].split('</address>')[0]
    price = list(soup.find_all (class_='list-card-price')) ## results, just need to re-format
    for i in range(len(price)):
        price[i] = int(str(price[i]).split('<div class="list-card-price">$')[1].split('</div>')[0].replace(',',''))
    details = list(soup.find_all("ul", class_="list-card-details")) ## long results, can parse beds, sqft, home_type
    beds = []
    sqft = []
    home_type = []

    for i in range(len(details)):
        z = str(details[i])
        beds.append(int(z.split('<ul class="list-card-details"><li class="">')[1].split('<')[0]))
        sqft.append(int(z.split('<abbr class="list-card-label"> <!-- -->ba</abbr></li><li class="">')[1].split('<')[0].replace(',', '')))
        home_type.append(z.split('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- ')[1].split(' ')[0])
    urls = []
    for link in soup.find_all('a', {'class': 'list-card-link'}, href=True):
        urls.append(link['href'])
    urls = list(set(urls))
    
    zillow_zestimate = []
    for i in range(len(urls)):
        link = urls[i]
        r = s.get(link, headers=req_headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        home_value = str(soup.find_all('span', {'class':'Text-c11n-8-53-2__sc-aiai24-0 fBcVfR'})[0])
        if not home_value:
            home_value = soup.select_one('.zestimate').text.split()[-1]
        else:
            home_value = int(home_value.split('<span class="Text-c11n-8-53-2__sc-aiai24-0 fBcVfR">$')[1].split('</span>')[0].replace(',',''))
        zillow_zestimate.append(home_value)
    
    df['Address'] = address
    df['Beds'] = beds
    df['SQFT'] = sqft
    df['Home_Type'] = home_type
    df['Price'] = price
    df['Link'] = urls
    df['Zestimate'] = zillow_zestimate
    
    return df
    
    

In [18]:
web_scrape(soup)

,Address,Beds,SQFT,Home_Type,Price,Link,Zestimate
0,"4091 Midrose Trl, Dallas, TX 75287",3,1476,House,359900,https://www.zillow.com/homedetails/9028-Enchan...,508084
1,"6205 Texana Way, Plano, TX 75074",3,1423,Townhouse,323200,https://www.zillow.com/homedetails/4091-Midros...,359903
2,"1719 River Oaks Dr, Allen, TX 75002",3,1730,House,399900,https://www.zillow.com/homedetails/4705-Newbri...,450536
3,"1808 Melton Dr, Carrollton, TX 75010",3,1630,House,393900,https://www.zillow.com/homedetails/8370-Hidden...,481000
4,"4908 Justin Dr, Plano, TX 75024",4,2081,House,446100,https://www.zillow.com/homedetails/1808-Melton...,404450
5,"8370 Hidden Spring Dr, Frisco, TX 75034",4,2086,House,444900,https://www.zillow.com/homedetails/1719-River-...,419500
6,"9028 Enchanted Ridge Dr, Plano, TX 75025",4,2283,House,470900,https://www.zillow.com/homedetails/6205-Texana...,334668
7,"4705 Newbridge Dr, McKinney, TX 75070",3,1945,House,419900,https://www.zillow.com/homedetails/1207-S-Coll...,325300
8,"1207 S College St, McKinney, TX 75069",3,1168,House,315900,https://www.zillow.com/homedetails/4908-Justin...,465800


In [7]:
zillow_zestimate = []
for i in range(len(urls)):
    link = urls[i]
    r = s.get(link, headers=req_headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    home_value = str(soup.find_all('span', {'class':'Text-c11n-8-53-2__sc-aiai24-0 fBcVfR'})[0])
    if not home_value:
        home_value = soup.select_one('.zestimate').text.split()[-1]
    else:
        home_value = int(home_value.split('<span class="Text-c11n-8-53-2__sc-aiai24-0 fBcVfR">$')[1].split('</span>')[0].replace(',',''))
    zillow_zestimate.append(home_value)

In [8]:
df['Zestimate'] = zillow_zestimate
df.head()

,Address,Beds,SQFT,Home_Type,Price,Link,Zestimate
0,"6205 Texana Way, Plano, TX 75074",3,1423,Townhouse,323200,https://www.zillow.com/homedetails/4600-Cedar-...,377742
1,"4091 Midrose Trl, Dallas, TX 75287",3,1476,House,359900,https://www.zillow.com/homedetails/4091-Midros...,359903
2,"3804 Ash Ln, McKinney, TX 75070",3,2448,House,430900,https://www.zillow.com/homedetails/12105-Chatt...,441717
3,"1719 River Oaks Dr, Allen, TX 75002",3,1730,House,399900,https://www.zillow.com/homedetails/8370-Hidden...,481000
4,"8370 Hidden Spring Dr, Frisco, TX 75034",4,2086,House,444900,https://www.zillow.com/homedetails/1719-River-...,419500


In [9]:
df.columns

Index(['Address', 'Beds', 'SQFT', 'Home_Type', 'Price', 'Link', 'Zestimate'], dtype='object')

In [ ]:
zillow_zestimate = []
for link in urls:
    r = s.get(link, headers=req_headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    home_value = str(soup.find_all('span', {'class':'Text-c11n-8-53-2__sc-aiai24-0 fBcVfR'})[0])
    if not home_value:
        home_value = soup.select_one('.zestimate').text.split()[-1]
    else:
        home_value = int(home_value.split('<span class="Text-c11n-8-53-2__sc-aiai24-0 fBcVfR">$')[1].split('</span>')[0].replace(',',''))
    zillow_zestimate.append(home_value)

In [ ]:
zillow_zestimate

In [ ]:
z = str(details[0])
beds = int(z.split('<ul class="list-card-details"><li class="">')[1].split('<')[0])
sqft = int(z.split('<abbr class="list-card-label"> <!-- -->ba</abbr></li><li class="">')[1].split('<')[0].replace(',', ''))
home_type = z.split('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- ')[1].split(' ')[0]

In [ ]:
beds = []
sqft = []
home_type = []

for i in range(len(details)):
    z = str(details[i])
    beds.append(int(z.split('<ul class="list-card-details"><li class="">')[1].split('<')[0]))
    sqft.append(int(z.split('<abbr class="list-card-label"> <!-- -->ba</abbr></li><li class="">')[1].split('<')[0].replace(',', '')))
    home_type.append(z.split('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">- ')[1].split(' ')[0])


In [ ]:
soups = []
#all for loops are pulling the specified variable using beautiful soup and inserting into said variable
for i in soup:
    address = soup.find_all (class_= 'list-card-addr') ## no results, can get from house url?
    price = list(soup.find_all (class_='list-card-price')) ## results, just need to re-format
    beds = list(soup.find_all("ul", class_="list-card-details")) ## long results, can be simplified
    details = soup.find_all ('div', {'class': 'list-card-details'}) ## no results
    home_type = soup.find_all ('div', {'class': 'list-card-footer'}) ## no results
    last_updated = soup.find_all ('div', {'class': 'list-card-top'}) ## no results
    brokerage = list(soup.find_all(class_= 'list-card-brokerage list-card-img-overlay',text=True)) ## no results
    ### Stopping point
    urls = []
    for link in soup.find_all('a', {'class': 'list-card-link'}, href=True):
        urls.append(link['href'])
    urls = list(set(urls))
    
    #create dataframe columns out of variables
    df['prices'] = price
    df['address'] = address
    df['beds'] = beds

#create empty url list
urls = []

#loop through url, pull the href and strip out the address tag
for link in soup.find_all("article"):
    href = link.find('a',class_="list-card-link")
    addresses = href.find('address')
    addresses.extract()
    urls.append(href)

#import urls into a links column
df['links'] = urls
df['links'] = df['links'].astype('str')

#remove html tags
df['links'] = df['links'].replace('<a class="list-card-link" href="', ' ', regex=True)
df['links'] = df['links'].replace('" tabindex="0"></a>', ' ', regex=True)


for i in soup1:
    address1 = soup1.find_all (class_= 'list-card-addr')
    price1 = list(soup1.find_all (class_='list-card-price'))
    beds1 = list(soup.find_all("ul", class_="list-card-details"))
    details1 = soup1.find_all ('div', {'class': 'list-card-details'})
    home_type1 = soup1.find_all ('div', {'class': 'list-card-footer'})
    last_updated1 = soup1.find_all ('div', {'class': 'list-card-top'})
    brokerage1 = list(soup1.find_all(class_= 'list-card-brokerage list-card-img-overlay',text=True))
    link1 = soup1.find_all (class_= 'list-card-link')

    #create dataframe columns out of variables
    df1['prices'] = price1
    df1['address'] = address1
    df1['beds'] = beds

#create empty url list
urls = []

#loop through url, pull the href and strip out the address tag
for link in soup1.find_all("article"):
    href = link.find('a',class_="list-card-link")
    addresses = href.find('address')
    addresses.extract()
    urls.append(href)

#import urls into a links column
df1['links'] = urls
df1['links'] = df1['links'].astype('str')

#remove html tags
df1['links'] = df1['links'].replace('<a class="list-card-link" href="', ' ', regex=True)
df1['links'] = df1['links'].replace('" tabindex="0"></a>', ' ', regex=True)

#append first two dataframes
df = df.append(df1, ignore_index = True) 

#create empty dataframes
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()
df5 = pd.DataFrame()
df6 = pd.DataFrame()
df7 = pd.DataFrame()
df8 = pd.DataFrame()
df9 = pd.DataFrame()

for i in soup2:
    soup = soup2
    address = soup.find_all (class_= 'list-card-addr')
    price = list(soup.find_all (class_='list-card-price'))
    beds = list(soup.find_all("ul", class_="list-card-details"))
    details = soup.find_all ('div', {'class': 'list-card-details'})
    home_type = soup.find_all ('div', {'class': 'list-card-footer'})
    last_updated = soup.find_all ('div', {'class': 'list-card-top'})
    brokerage = list(soup.find_all(class_= 'list-card-brokerage list-card-img-overlay',text=True))
    link = soup.find_all (class_= 'list-card-link')
    
    #create dataframe columns out of variables
    df2['prices'] = price
    df2['address'] = address
    df2['beds'] = beds

#create empty url list
urls = []

#loop through url, pull the href and strip out the address tag
for link in soup2.find_all("article"):
    href = link.find('a',class_="list-card-link")
    addresses = href.find('address')
    addresses.extract()
    urls.append(href)

#import urls into a links column
df2['links'] = urls
df2['links'] = df2['links'].astype('str')

#remove html tags
df2['links'] = df2['links'].replace('<a class="list-card-link" href="', ' ', regex=True)
df2['links'] = df2['links'].replace('" tabindex="0"></a>', ' ', regex=True)

    
for i in soup3:
    soup = soup3
    address1 = soup.find_all (class_= 'list-card-addr')
    price1 = list(soup.find_all (class_='list-card-price'))
    beds1 = list(soup.find_all("ul", class_="list-card-details"))
    details1 = soup.find_all ('div', {'class': 'list-card-details'})
    home_type1 = soup.find_all ('div', {'class': 'list-card-footer'})
    last_updated1 = soup.find_all ('div', {'class': 'list-card-top'})
    brokerage1 = list(soup.find_all(class_= 'list-card-brokerage list-card-img-overlay',text=True))
    link1 = soup.find_all (class_= 'list-card-link')

    #create dataframe columns out of variables
    df3['prices'] = price1
    df3['address'] = address1
    df3['beds'] = beds

#create empty url list
urls = []

#loop through url, pull the href and strip out the address tag
for link in soup3.find_all("article"):
    href = link.find('a',class_="list-card-link")
    addresses = href.find('address')
    addresses.extract()
    urls.append(href)

#import urls into a links column
df3['links'] = urls
df3['links'] = df3['links'].astype('str')

#remove html tags
df3['links'] = df3['links'].replace('<a class="list-card-link" href="', ' ', regex=True)
df3['links'] = df3['links'].replace('" tabindex="0"></a>', ' ', regex=True)

for i in soup4:
    soup = soup4
    address1 = soup.find_all (class_= 'list-card-addr')
    price1 = list(soup.find_all (class_='list-card-price'))
    beds1 = list(soup.find_all("ul", class_="list-card-details"))
    details1 = soup.find_all ('div', {'class': 'list-card-details'})
    home_type1 = soup.find_all ('div', {'class': 'list-card-footer'})
    last_updated1 = soup.find_all ('div', {'class': 'list-card-top'})
    brokerage1 = list(soup.find_all(class_= 'list-card-brokerage list-card-img-overlay',text=True))
    link1 = soup.find_all (class_= 'list-card-link')

    #create dataframe columns out of variables
    df4['prices'] = price1
    df4['address'] = address1
    df4['beds'] = beds

#create empty url list
urls = []

#loop through url, pull the href and strip out the address tag
for link in soup4.find_all("article"):
    href = link.find('a',class_="list-card-link")
    addresses = href.find('address')
    addresses.extract()
    urls.append(href)

#import urls into a links column
df4['links'] = urls
df4['links'] = df4['links'].astype('str')

#remove html tags
df4['links'] = df4['links'].replace('<a class="list-card-link" href="', ' ', regex=True)
df4['links'] = df4['links'].replace('" tabindex="0"></a>', ' ', regex=True)

for i in soup5:
    soup = soup5
    address1 = soup.find_all (class_= 'list-card-addr')
    price1 = list(soup.find_all (class_='list-card-price'))
    beds1 = list(soup.find_all("ul", class_="list-card-details"))
    details1 = soup.find_all ('div', {'class': 'list-card-details'})
    home_type1 = soup.find_all ('div', {'class': 'list-card-footer'})
    last_updated1 = soup.find_all ('div', {'class': 'list-card-top'})
    brokerage1 = list(soup.find_all(class_= 'list-card-brokerage list-card-img-overlay',text=True))
    link1 = soup.find_all (class_= 'list-card-link')

    #create dataframe columns out of variables
    df5['prices'] = price1
    df5['address'] = address1
    df5['beds'] = beds

#create empty url list
urls = []

#loop through url, pull the href and strip out the address tag
for link in soup5.find_all("article"):
    href = link.find('a',class_="list-card-link")
    addresses = href.find('address')
    addresses.extract()
    urls.append(href)

#import urls into a links column
df5['links'] = urls
df5['links'] = df5['links'].astype('str')

#remove html tags
df5['links'] = df5['links'].replace('<a class="list-card-link" href="', ' ', regex=True)
df5['links'] = df5['links'].replace('" tabindex="0"></a>', ' ', regex=True)
    
for i in soup6:
    soup = soup6
    address1 = soup.find_all (class_= 'list-card-addr')
    price1 = list(soup.find_all (class_='list-card-price'))
    beds1 = list(soup.find_all("ul", class_="list-card-details"))
    details1 = soup.find_all ('div', {'class': 'list-card-details'})
    home_type1 = soup.find_all ('div', {'class': 'list-card-footer'})
    last_updated1 = soup.find_all ('div', {'class': 'list-card-top'})
    brokerage1 = list(soup.find_all(class_= 'list-card-brokerage list-card-img-overlay',text=True))
    link1 = soup.find_all (class_= 'list-card-link')

    #create dataframe columns out of variables
    df6['prices'] = price1
    df6['address'] = address1
    df6['beds'] = beds

#create empty url list
urls = []

#loop through url, pull the href and strip out the address tag
for link in soup6.find_all("article"):
    href = link.find('a',class_="list-card-link")
    addresses = href.find('address')
    addresses.extract()
    urls.append(href)

#import urls into a links column
df6['links'] = urls
df6['links'] = df6['links'].astype('str')

#remove html tags
df6['links'] = df6['links'].replace('<a class="list-card-link" href="', ' ', regex=True)
df6['links'] = df6['links'].replace('" tabindex="0"></a>', ' ', regex=True)
    
for i in soup7:
    soup = soup7
    address1 = soup.find_all (class_= 'list-card-addr')
    price1 = list(soup.find_all (class_='list-card-price'))
    beds1 = list(soup.find_all("ul", class_="list-card-details"))
    details1 = soup.find_all ('div', {'class': 'list-card-details'})
    home_type1 = soup.find_all ('div', {'class': 'list-card-footer'})
    last_updated1 = soup.find_all ('div', {'class': 'list-card-top'})
    brokerage1 = list(soup.find_all(class_= 'list-card-brokerage list-card-img-overlay',text=True))
    link1 = soup.find_all (class_= 'list-card-link')

    #create dataframe columns out of variables
    df7['prices'] = price1
    df7['address'] = address1
    df7['beds'] = beds

#create empty url list
urls = []

#loop through url, pull the href and strip out the address tag
for link in soup7.find_all("article"):
    href = link.find('a',class_="list-card-link")
    addresses = href.find('address')
    addresses.extract()
    urls.append(href)

#import urls into a links column
df7['links'] = urls
df7['links'] = df7['links'].astype('str')

#remove html tags
df7['links'] = df7['links'].replace('<a class="list-card-link" href="', ' ', regex=True)
df7['links'] = df7['links'].replace('" tabindex="0"></a>', ' ', regex=True)
    
for i in soup8:
    soup = soup8
    address1 = soup.find_all (class_= 'list-card-addr')
    price1 = list(soup.find_all (class_='list-card-price'))
    beds1 = list(soup.find_all("ul", class_="list-card-details"))
    details1 = soup.find_all ('div', {'class': 'list-card-details'})
    home_type1 = soup.find_all ('div', {'class': 'list-card-footer'})
    last_updated1 = soup.find_all ('div', {'class': 'list-card-top'})
    brokerage1 = list(soup.find_all(class_= 'list-card-brokerage list-card-img-overlay',text=True))
    link1 = soup.find_all (class_= 'list-card-link')

    #create dataframe columns out of variables
    df8['prices'] = price1
    df8['address'] = address1
    df8['beds'] = beds

#create empty url list
urls = []

#loop through url, pull the href and strip out the address tag
for link in soup8.find_all("article"):
    href = link.find('a',class_="list-card-link")
    addresses = href.find('address')
    addresses.extract()
    urls.append(href)

#import urls into a links column
df8['links'] = urls
df8['links'] = df8['links'].astype('str')

#remove html tags
df8['links'] = df8['links'].replace('<a class="list-card-link" href="', ' ', regex=True)
df8['links'] = df8['links'].replace('" tabindex="0"></a>', ' ', regex=True)

for i in soup9:
    soup = soup9
    address1 = soup.find_all (class_= 'list-card-addr')
    price1 = list(soup.find_all (class_='list-card-price'))
    beds1 = list(soup.find_all("ul", class_="list-card-details"))
    details1 = soup.find_all ('div', {'class': 'list-card-details'})
    home_type1 = soup.find_all ('div', {'class': 'list-card-footer'})
    last_updated1 = soup.find_all ('div', {'class': 'list-card-top'})
    brokerage1 = list(soup.find_all(class_= 'list-card-brokerage list-card-img-overlay',text=True))
    link1 = soup.find_all (class_= 'list-card-link')

    #create dataframe columns out of variables
    df9['prices'] = price1
    df9['address'] = address1
    df9['beds'] = beds

#create empty url list
urls = []

#loop through url, pull the href and strip out the address tag
for link in soup9.find_all("article"):
    href = link.find('a',class_="list-card-link")
    addresses = href.find('address')
    addresses.extract()
    urls.append(href)

#import urls into a links column
df9['links'] = urls
df9['links'] = df9['links'].astype('str')

#remove html tags
df9['links'] = df9['links'].replace('<a class="list-card-link" href="', ' ', regex=True)
df9['links'] = df9['links'].replace('" tabindex="0"></a>', ' ', regex=True)

df = df.append(df2, ignore_index = True) 
df = df.append(df3, ignore_index = True) 
df = df.append(df4, ignore_index = True) 
df = df.append(df5, ignore_index = True) 
df = df.append(df6, ignore_index = True) 
df = df.append(df7, ignore_index = True) 
df = df.append(df8, ignore_index = True) 
df = df.append(df9, ignore_index = True) 

#convert columns to str
df['prices'] = df['prices'].astype('str')
df['address'] = df['address'].astype('str')
df['beds'] = df['beds'].astype('str')

#remove html tags
df['prices'] = df['prices'].replace('<div class="list-card-price">', ' ', regex=True)
df['address'] = df['address'].replace('<address class="list-card-addr">', ' ', regex=True)
df['prices'] = df['prices'].replace('</div>', ' ', regex=True)
df['address'] = df['address'].replace('</address>', ' ', regex=True)
df['prices'] = df['prices'].str.replace(r'\D', '')

#remove html tags from beds column
df['beds'] = df['beds'].replace('<ul class="list-card-details"><li>', ' ', regex=True)
df['beds'] = df['beds'].replace('<abbr class="list-card-label"> <!-- -->bds</abbr></li><li>', ' ', regex=True)
df['beds'] = df['beds'].replace('<abbr class="list-card-label"> <!-- -->ba</abbr></li><li>', ' ', regex=True)
df['beds'] = df['beds'].replace('<abbr class="list-card-label"> <!-- -->bd</abbr></li><li>', ' ', regex=True)
df['beds'] = df['beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li></ul>', ' ', regex=True)
df['beds'] = df['beds'].replace('Studio</li><li>', '0 ', regex=True)

#split beds column into beds, bath and sq_feet
df[['beds','baths','sq_feet']] = df.beds.str.split(expand=True)

#remove commas from sq_feet and convert to float
df.replace(',','', regex=True, inplace=True)

#drop nulls
df = df[(df['prices'] != '') & (df['prices']!= ' ')]

#convert column to float
df['prices'] = df['prices'].astype('float')
# d['sq_feet'] = df['sq_feet'].astype('float')

#remove spaces from link column
df['links'] = df.links.str.replace(' ','')

print('The column datatypes are:')
print(df.dtypes)
print('The dataframe shape is:', df.shape)

#rearrange the columns
df = df[['prices', 'address', 'links', 'beds', 'baths', 'sq_feet']]

# df

#calculate the zestimate and insert into a dataframe
zillow_zestimate = []
for link in df['links']:
    r = s.get(link, headers=req_headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    home_value = soup.select_one('h4:contains("Home value")')
    if not home_value:
        home_value = soup.select_one('.zestimate').text.split()[-1]
    else:
        home_value = home_value.find_next('p').get_text(strip=True)
    zillow_zestimate.append(home_value)

cols=['zestimate']
zestimate_result = pd.DataFrame(zillow_zestimate, columns=cols)
# zestimate_result

#convert zestimate column to float, and remove , and $
zestimate_result['zestimate'] = zestimate_result['zestimate'].str.replace('$','')
zestimate_result['zestimate'] = zestimate_result['zestimate'].str.replace('/mo','')
zestimate_result['zestimate'] = zestimate_result['zestimate'].str.replace(',','')

#covert rows with non zestimate to 0
def non_zestimate(zestimate_result):
    if len(zestimate_result['zestimate']) > 20:
        return '0'
    elif len(zestimate_result['zestimate']) < 5:
        return '0'
    else:
        return zestimate_result['zestimate']

zestimate_result['zestimate'] = zestimate_result.apply(non_zestimate,axis=1)

# zestimate_result

#concat zestimate dataframe and original df
df = pd.concat([df, zestimate_result], axis=1)
df['zestimate'] = df['zestimate'].astype('float')

#create best deal column and sort by best_deal
df ['best_deal'] = df['prices'] - df['zestimate']
df = df.sort_values(by='best_deal')

df